In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_path = '/content/drive/My Drive/dataset'


In [3]:
import os
import numpy as np
from PIL import Image
from skimage.feature import hog
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

In [4]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        if not os.path.isdir(class_path):
            continue
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize to a fixed size

            # Extract HOG features
            img_array = np.array(img)
            hog_features = hog(img_array, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

            images.append(hog_features)
            labels.append(class_folder)
    return images, labels

In [5]:
train_folder = os.path.join(dataset_path, 'train')
test_folder = os.path.join(dataset_path, 'test')

In [6]:
X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)

In [7]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [8]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [9]:
# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
# Define the ANN model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.5928 - loss: 0.7095 - val_accuracy: 0.0000e+00 - val_loss: 0.9081
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6743 - loss: 0.6147 - val_accuracy: 0.7589 - val_loss: 0.5605
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6511 - loss: 0.6087 - val_accuracy: 0.5625 - val_loss: 0.6647
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7890 - loss: 0.5132 - val_accuracy: 0.7232 - val_loss: 0.5315
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7533 - loss: 0.5002 - val_accuracy: 0.4375 - val_loss: 0.8504
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8338 - loss: 0.3959 - val_accuracy: 0.4286 - val_loss: 0.8981
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8692 - loss: 0.3442 - val_accuracy: 0.3929 - val_loss: 1.1596
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8518 - loss: 0.3445 - val_accuracy: 0.4286

In [13]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [14]:
# KPIs
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [15]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print(classification_report(y_test_classes, y_pred_classes))

Accuracy: 52.86%
Precision: 53.17%
Recall: 52.86%
F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.52      0.69      0.59        70
           1       0.54      0.37      0.44        70

    accuracy                           0.53       140
   macro avg       0.53      0.53      0.52       140
weighted avg       0.53      0.53      0.52       140

